In [ ]:
import sys
sys.path.append("..")

In [ ]:
import numpy as np
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import seaborn as sns
import matplotlib as mpl

import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
from paths import file_ex_ocn_ctrl, path_results
from timeseries import IterateOutputCESM
from xr_regression import xr_linear_trend

In [ ]:
ds = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)

In [ ]:
from maps import map_ocn_robinson

In [ ]:
ds.UVEL[0,0,:,:]

In [ ]:
da = ds.UVEL[0,0,:,:].where(ds.REGION_MASK>0)
cmap = 'RdBu'
minv, maxv = -50, 50
label= 'zonal velocity component [cm/s]'
filename = f'{path_results}/CURRENTS/UVEL_global_map'
map_ocn_robinson(da, cmap=cmap, minv=minv, maxv=maxv, label=label, filename=filename)

In [ ]:
fig = plt.figure(figsize=(8,5))
ax  = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_position([.02,.05,.66,.88])
cax, kw = mpl.colorbar.make_axes(ax,location='bottom',pad=0.03,shrink=0.8)
im = ax.pcolormesh(ds.TLONG[550:800,1700:1900],
                   ds.TLAT[550:800,1700:1900],
                   ds.UVEL[0,0,550:800,1700:1900],
                   cmap='RdBu',
                   vmin=-50, vmax=50,
                   transform=ccrs.PlateCarree() )
ax.coastlines()
ax.plot([70,70],[-48,-38], c='g', lw=3)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.xlabels_bottom = False
gl.ylabels_right = False
gl.xlabel_style = {'size': 14}
gl.ylabel_style = {'size': 14}
# gl.xlabel_style = {'color': 'red', 'weight': 'bold'}

cbar = fig.colorbar(im, cax=cax, extend='both', **kw)
cbar.ax.tick_params(labelsize=14)
label = cbar.set_label('zonal velocity [cm/s]', size=16)

ax2 = fig.add_subplot(1, 2, 2)
ax2.set_position([.68,.21,.3,.72])
ax2.tick_params(labelsize=14)
ax2.set_xlabel('UVEL @ transect', fontsize=16)
ax2.axvline(0, c='k', lw=.5)
ax2.plot(ds.UVEL[0,0,630:770,1800], ds.TLAT[630:770,1800], label='snapshot')
ax2.plot(jet_ctrl[:,:].mean(dim='time'), jet_ctrl.TLAT, lw=3, label='mean')
ax2.legend()
ax2.set_xticks(np.arange(-25,76,25))

plt.savefig(f'{path_results}/CURRENTS/UVEL_Kerguelen_detail')

## Hovmoeller diagram

In [ ]:
# %%time
# for run in ['ctrl', 'rcp']:
#     for y, m, file in IterateOutputCESM(domain='ocn', run=run, tavg='monthly'):
#         ds = xr.open_dataset(file, decode_times=False)
#         if m==1 and y==IterateOutputCESM(domain='ocn', run=run, tavg='monthly').year:
#             ds_new = ds.UVEL[0,0,630:770,1800]
#         else:
#             ds_new = xr.concat([ds_new, ds.UVEL[0,0,630:770,1800]], dim='time')
#     ds_new.to_netcdf(path=f'{path_results}/currents/Kerguelen_jet_{run}.nc', mode='w')

In [ ]:
jet_ctrl = xr.open_dataarray(f'{path_results}/CURRENTS/Kerguelen_jet_ctrl.nc', decode_times=False)
jet_rcp  = xr.open_dataarray(f'{path_results}/CURRENTS/Kerguelen_jet_rcp.nc' , decode_times=False)

In [ ]:
jet_ctrl

In [ ]:
jet_rcp.rolling(time=12, center=True).mean().plot()

In [ ]:
jet_ctrl_mean = jet_ctrl.rolling(time=120, center=True).mean()
jet_rcp_mean  = jet_rcp.rolling( time=120, center=True).mean()

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(8,5), sharey=True)

ax[0].set_position([.1,.13,.4,.85])
ax[1].set_position([.55,.13,.4,.85])


cax, kw = mpl.colorbar.make_axes(ax[1],location='right',pad=0.03,shrink=0.9)

im = ax[0].pcolormesh(jet_ctrl_mean.time/365,
                 jet_ctrl_mean.ULAT,
                 jet_ctrl_mean.values.T)
ax[1].pcolormesh(jet_rcp_mean.time/365,
                 jet_rcp_mean.ULAT,
                 jet_rcp_mean.values.T)

cbar = fig.colorbar(im, cax=cax, extend='both', **kw)
cbar.ax.tick_params(labelsize=14)
label = cbar.set_label('zonal velocity [cm/s]', size=16)

ax[0].set_ylabel('latitude', fontsize=16)
for i in range(2):
    ax[i].tick_params(labelsize=14)
    ax[i].set_xlabel('time [model years]', fontsize=16)
# plt.tight_layout()
plt.savefig(f'{path_results}/CURRENTS/Kerguelen_jet_Hovmoeller')

In [ ]:
from xr_regression import xr_linear_trend

In [ ]:
jet_ctrl

In [ ]:
jet_trend_rcp = xr_linear_trend(jet_rcp)

In [ ]:
# for t in range(len(jet_ctrl.time)):
#     plt.plot(jet_ctrl[t,:], jet_ctrl.TLAT, c='k', alpha=.5)
# plt.plot(jet_ctrl[:,:].mean(dim='time'), jet_ctrl.TLAT, lw=3)
fig = plt.figure(figsize=(8,5))



# Trends

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(8,5))

ax[0].tick_params(labelsize=14)
ax[0].plot(jet_ctrl[:,:].mean(dim='time'), jet_ctrl.TLAT, lw=2, label='CTRL')
ax[0].plot(jet_rcp[:,:].mean(dim='time') , jet_rcp.TLAT , lw=2, label='RCP')
# ax[0].plot(jet_rcp[0:120,:].mean(dim='time')  , jet_rcp.TLAT, lw=3, ls='--')
# ax[0].plot(jet_rcp[-121:-1,:].mean(dim='time'), jet_rcp.TLAT, lw=3, ls='--')
ax[0].set_ylabel('latitude [deg N]', fontsize=16)
ax[0].set_xlabel('mean UVEL [cm/s]')
ax[0].legend(fontsize=16)

ax[1].tick_params(labelsize=14)
ax[1].axvline(0, c='k', lw=.5)
ax[1].plot(100*365*xr_linear_trend(jet_ctrl), jet_ctrl.TLAT, lw=2, label='CTRL')
ax[1].plot(100*365*xr_linear_trend(jet_rcp) , jet_rcp.TLAT , lw=2, label='RCP')
ax[1].set_xlabel('UVEL trend [cm/s/century]', fontsize=16)

plt.tight_layout()
plt.savefig(f'{path_results}/CURRENTS/Kerguelen_jet_UVEL_trends')